In [34]:
from tinydb import TinyDB, Query
from datetime import datetime

Record = Query()

db = TinyDB('trailing_take_profit.json')
db_logs = TinyDB('logs.json')


In [9]:
STOP_LOSS = 0.004       # i.e. default = 0.4%
TAKE_PROFIT = 0.01      # i.e. default = 1%
ENABLE_STOP_LOSS = True
ENABLE_TAKE_PROFIT = True


In [70]:
def updateOrCreate(coin_record):
    found = len(db.search(Record.symbol == coin_record.get('symbol')))
    # print(f"found {found}")
    if found > 0:
        db.update(coin_record, Record.symbol == coin_record.get('symbol'))
    else:
        db.insert(coin_record)
        

def getCoinRecord(symbol):
    arr = db.search(Record.symbol == symbol)
#     print(f'arr {arr}')
    if(len(arr) <= 0):
        return {}
    return arr[0]

def removeCoinRecord(symbol):
    try:
        db.remove(Record.symbol == symbol)
        return True
    except Exception as err:
        print(f"err {err}")
        return False
    

In [71]:
symbol = "BTCUSDT"
current_price = 12000
buy_price = 10000
take_profit = 11000
tralling_take_profit = 0.05
stop_order = 0


coin_record = {"symbol": symbol,"buy_price": buy_price, "take_profit": take_profit, "tralling_take_profit": tralling_take_profit, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# coin_record = {"symbol": symbol,"stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
print(coin_record)
updateOrCreate(coin_record)

{'symbol': 'BTCUSDT', 'buy_price': 10000, 'take_profit': 11000, 'tralling_take_profit': 0.05, 'stop_order': 0, 'createdAt': '25/08/2021 11:57:43'}


In [72]:
# def do_stop_order(symbol, )
coin_record = getCoinRecord(symbol)
coin_record

{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 0,
 'createdAt': '25/08/2021 11:57:43'}

In [73]:
def trailing_take_profit(symbol, current_price):
    coin_record = getCoinRecord(symbol)
    if(coin_record.get('symbol')):
        if(current_price < coin_record.get('take_profit')):
            return False
        else:
            
            current_stop_order = coin_record.get('stop_order')
            if(current_price <= current_stop_order):
                return True
            
            new_stop_order = current_price*(1-coin_record.get('tralling_take_profit'))
            coin_record = {"symbol": symbol,"stop_order": max(current_stop_order, new_stop_order) , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
            print(coin_record)
            updateOrCreate(coin_record)
        
    return False

In [74]:
# buy btc 10000
# ENABLE_TAKE_PROFIT
enable_take_profit = trailing_take_profit("BTCUSDT", 10000)
print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT")
coin_record

enable_take_profit False


{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 0,
 'createdAt': '25/08/2021 11:57:43'}

In [81]:
symbol = "BTCUSDT"
current_price = 12000
buy_price = 10000
take_profit = 11000
tralling_take_profit = 0.05
stop_order = 0


coin_record = {"symbol": symbol,"buy_price": buy_price, "take_profit": take_profit, "tralling_take_profit": tralling_take_profit, "stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
# coin_record = {"symbol": symbol,"stop_order": stop_order , "createdAt": datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
print(coin_record)
updateOrCreate(coin_record)

{'symbol': 'BTCUSDT', 'buy_price': 10000, 'take_profit': 11000, 'tralling_take_profit': 0.05, 'stop_order': 0, 'createdAt': '25/08/2021 12:01:17'}


In [82]:
enable_take_profit = trailing_take_profit("BTCUSDT", 10000)
print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT")
coin_record

enable_take_profit False


{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 0,
 'createdAt': '25/08/2021 12:01:17'}

In [83]:
enable_take_profit = trailing_take_profit("BTCUSDT", 10500)
print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT")
coin_record


enable_take_profit False


{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 0,
 'createdAt': '25/08/2021 12:01:17'}

In [84]:
enable_take_profit = trailing_take_profit("BTCUSDT", 12000)
print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT")
coin_record

{'symbol': 'BTCUSDT', 'stop_order': 11400.0, 'createdAt': '25/08/2021 12:01:36'}
enable_take_profit False


{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 11400.0,
 'createdAt': '25/08/2021 12:01:36'}

In [85]:
enable_take_profit = trailing_take_profit("BTCUSDT", 11000)
print(f"enable_take_profit {enable_take_profit}")
coin_record = getCoinRecord("BTCUSDT")
coin_record

enable_take_profit True


{'symbol': 'BTCUSDT',
 'buy_price': 10000,
 'take_profit': 11000,
 'tralling_take_profit': 0.05,
 'stop_order': 11400.0,
 'createdAt': '25/08/2021 12:01:36'}

In [87]:
removeCoinRecord("BTCUSDT")
coin_record = getCoinRecord("BTCUSDT")
coin_record

{}